다른 사람들은 우리 프로젝트와 관련이 있는 분류 예제를 가져올것이라 생각.
나는 우리 문제와는 거리가 멀지만 알아두면 좋은 분류 문제를 가져옴.

### 공부 시간에 따른 시험 합격 예측 (로지스틱 회귀)

이 예제는 **공부한 시간**이라는 독립 변수(X)를 사용하여, \*\*시험 합격 여부(0: 불합격, 1: 합격)\*\*라는 종속 변수(y)를 예측하는 이진 분류 모델을 만듭니다.  
`scikit-learn` 라이브러리를 사용하여 로지스틱 회귀 모델을 간단하게 구현하고, `matplotlib`으로 결과를 시각화합니다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# 1. 예제 데이터 생성
# X: 공부 시간 (hours)
X = np.array([[0.5], [1.0], [1.5], [2.0], [2.5], [3.0], [3.5], [4.0], [4.5], [5.0], [5.5], [6.0]])
# y: 합격 여부 (0: Fail, 1: Pass)
y = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [ ]:
X, y, np.shape(X), np.shape(y), type(X), type(y)

In [ ]:
# 2. 로지스틱 회귀 모델 생성 및 학습
model = LogisticRegression()
model.fit(X, y)

In [ ]:
model

In [ ]:
# 3. 결과 시각화
plt.figure(figsize=(8, 5))

# 원본 데이터 점 찍기 (Scatter plot)
plt.scatter(X, y, color='blue', zorder=20, label='Actual Data')

# 로지스틱 회귀 곡선 그리기
X_test = np.linspace(-1, 7, 300).reshape(-1, 1) # 예측을 위한 x값 범위
y_prob = model.predict_proba(X_test)[:, 1] # 클래스 1(합격)에 대한 확률 예측
plt.plot(X_test, y_prob, color='red', linewidth=3, label='Logistic Regression Curve')

# 결정 경계선(확률=0.5) 표시
decision_boundary = -model.intercept_ / model.coef_[0]
plt.axvline(x=decision_boundary, color='green', linestyle='--', label='Decision Boundary')

# 그래프 제목 및 라벨 설정
plt.title('Study Hours vs. Pass/Fail') # 그래프 제목
plt.xlabel('Hours Studied') # x축 이름
plt.ylabel('Probability of Passing') # y축 이름
plt.yticks([0.0, 0.5, 1.0])
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
X_test, np.shape(X_test), type(X_test)

In [ ]:
y_prob, np.shape(y_prob), type(y_prob)

In [ ]:
decision_boundary

In [ ]:
-model.intercept_ 

In [ ]:
model.coef_[0]

In [ ]:
# 4. 새로운 데이터 예측
# 예를 들어 3.7시간과 3.8시간 공부했을 때의 합격 확률은?
new_hours = np.array([[3.7], [3.8]])
predicted_proba = model.predict_proba(new_hours)

print(f"결정 경계(Decision Boundary)는 약 {decision_boundary[0]:.2f} 시간입니다.")
print(f"3.7시간 공부했을 때 합격 확률: {predicted_proba[0][1]:.2%}")
print(f"3.8시간 공부했을 때 합격 확률: {predicted_proba[1][1]:.2%}")

## From Scratch
Numpy 라이브러리만을 사용하여 로지스틱 회귀의 핵심 알고리즘을 구현합니다.  
모델은 **경사 하강법(Gradient Descent)**을 통해 데이터에 가장 잘 맞는 최적의 가중치(Weight)를 점진적으로 찾아 나갑니다.  
이 과정은 마치 안개가 낀 산에서 가장 낮은 지점을 찾기 위해 계속해서 경사가 가파른 쪽으로 한 걸음씩 내려가는 것과 같습니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# 1. 시그모이드 함수 정의
# 모든 입력값을 0과 1 사이의 확률 값으로 변환하는 함수
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# 2. 파라미터(가중치) 초기화 및 데이터 준비
# X: 공부 시간 (hours)
X_data = np.array([[0.5], [1.0], [1.5], [2.0], [2.5], [3.0], [3.5], [4.0], [4.5], [5.0], [5.5], [6.0]])
# y: 합격 여부 (0: Fail, 1: Pass)
y = np.array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]).reshape(-1, 1) # y를 열벡터로 변환

In [2]:
X_data, np.shape(X_data), type(X_data)

(array([[0.5],
        [1. ],
        [1.5],
        [2. ],
        [2.5],
        [3. ],
        [3.5],
        [4. ],
        [4.5],
        [5. ],
        [5.5],
        [6. ]]),
 (12, 1),
 numpy.ndarray)

In [3]:
y, np.shape(y), type(y)

(array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [1],
        [1],
        [1],
        [1],
        [1]]),
 (12, 1),
 numpy.ndarray)

In [4]:
# 입력 데이터(X)에 편향(bias)을 위한 1의 열을 추가합니다.
# 이는 y = wx + b 형태의 수식에서 y절편 b를 학습하기 위함입니다.
X = np.c_[np.ones((len(X_data), 1)), X_data] # np.ones((12, 1))은 12행 1열의 1로 채워진 행렬을 생성합니다.

왜 편향을 추가하는 이유
편향(bias) 항을 추가하는 이유는 모델이 데이터의 중심을 더 잘 맞출 수 있도록 하기 위함입니다.  
편향 항이 없으면 모델은 원점을 지나야 하므로 데이터의 분포를 제대로 반영하지 못할 수 있습니다.

ex. y = wx + b (b는 편향 항)
이때 b가 없다면 모델은 y = wx 형태가 되어 원점을 지나야 합니다.  
편향 항 b가 있으면 모델이 데이터의 중심을 더 잘 맞출 수 있습니다.

In [5]:
len(X_data)

12

In [6]:
np.c_[np.ones((12, 1)), X_data] # X_data 첫번쨰 열에 편향(bias, 1) 추가

array([[1. , 0.5],
       [1. , 1. ],
       [1. , 1.5],
       [1. , 2. ],
       [1. , 2.5],
       [1. , 3. ],
       [1. , 3.5],
       [1. , 4. ],
       [1. , 4.5],
       [1. , 5. ],
       [1. , 5.5],
       [1. , 6. ]])

In [7]:
X, np.shape(X), type(X)

(array([[1. , 0.5],
        [1. , 1. ],
        [1. , 1.5],
        [1. , 2. ],
        [1. , 2.5],
        [1. , 3. ],
        [1. , 3.5],
        [1. , 4. ],
        [1. , 4.5],
        [1. , 5. ],
        [1. , 5.5],
        [1. , 6. ]]),
 (12, 2),
 numpy.ndarray)

In [8]:
# 가중치(weights)를 0으로 초기화합니다. [b, w] 형태가 됩니다.
weights = np.zeros((X.shape[1], 1)) # np.zeros((2, 1))과 동일
weights, np.shape(weights), type(weights)

(array([[0.],
        [0.]]),
 (2, 1),
 numpy.ndarray)

In [9]:
z = np.dot(X, weights) # X와 weights의 행렬 곱
print(f"첫번째 z 값:{z}, shape: {np.shape(z)}")
h = sigmoid(z) # 시그모이드 함수 적용
print(f"첫번째 h 값:{h}, shape: {np.shape(h)}")

첫번째 z 값:[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape: (12, 1)
첫번째 h 값:[[0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]
 [0.5]], shape: (12, 1)


In [10]:
error = h - y # 예측값(h)과 실제값(y)의 차이
print(f"첫번째 error 값:{error}, shape: {np.shape(error)}")

첫번째 error 값:[[ 0.5]
 [ 0.5]
 [ 0.5]
 [ 0.5]
 [ 0.5]
 [ 0.5]
 [-0.5]
 [-0.5]
 [-0.5]
 [-0.5]
 [-0.5]
 [-0.5]], shape: (12, 1)


In [11]:
m = len(y) # 데이터 개수
m

12

In [12]:
gradient = np.dot(X.T, error) / m
gradient

array([[ 0.  ],
       [-0.75]])

In [14]:
X.T, np.shape(X.T), type(X.T)

(array([[1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
        [0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. ]]),
 (2, 12),
 numpy.ndarray)

In [ ]:
learning_rate = 0.1
weights -= learning_rate * gradient
weights

In [ ]:
cost = -np.mean(y * np.log(h) + (1 - y) * np.log(1 - h))
cost

In [ ]:
# 3. 하이퍼파라미터 설정
learning_rate = 0.1  # 학습률: 한 걸음에 얼마나 이동할지 결정
iterations = 100000 # 학습 반복 횟수

# 4. 경사 하강법(Gradient Descent)을 이용한 모델 학습
m = len(y) # 데이터 개수

for i in range(iterations):
    # 예측값 계산 (가설 함수)
    z = np.dot(X, weights)
    h = sigmoid(z)
    
    # 오차(error) 계산
    error = h - y
    
    # 경사(gradient) 계산: 비용 함수를 가중치로 미분한 결과
    gradient = np.dot(X.T, error) / m
    
    # 가중치 업데이트
    weights -= learning_rate * gradient
    
    # 일정 주기마다 비용(Cost) 출력 (학습이 잘 되는지 확인용)
    if i % 10000 == 0:
        # 비용 함수(로그 손실) 계산
        cost = -np.mean(y * np.log(h) + (1 - y) * np.log(1 - h))
        print(f"Iteration {i}: Cost = {cost:.4f}")

print("\n** 학습 완료! **")
print(f"최종 가중치(weights): \n b (bias) = {weights[0][0]:.4f} \n w (slope) = {weights[1][0]:.4f}")


# 5. 결과 시각화
plt.figure(figsize=(8, 5))
plt.scatter(X_data, y, color='blue', zorder=20, label='Actual Data')

# 학습된 S자 곡선 그리기
X_test = np.linspace(-1, 7, 300)
X_test_with_bias = np.c_[np.ones((len(X_test), 1)), X_test]
y_prob = sigmoid(np.dot(X_test_with_bias, weights))
plt.plot(X_test, y_prob, color='red', linewidth=3, label='Learned Curve')

# 결정 경계선 계산 및 표시 (z = w*x + b = 0 이 되는 지점)
decision_boundary = -weights[0][0] / weights[1][0]
plt.axvline(x=decision_boundary, color='green', linestyle='--', label='Decision Boundary')

# 그래프 제목 및 라벨 설정
plt.title('Study Hours vs. Pass/Fail (Manual Implementation)') # 그래프 제목
plt.xlabel('Hours Studied') # x축 이름
plt.ylabel('Probability of Passing') # y축 이름
plt.legend()
plt.grid(True)
plt.show()

print(f"계산된 결정 경계는 약 {decision_boundary:.2f} 시간입니다.")